In [0]:
from pyspark.sql.functions import *

# Load logs
df = spark.read.option("multiline", "true").json("dbfs:/FileStore/logs.json")

# Convert timestamp to date
df = df.withColumn("date", to_date("timestamp"))

# Filter only ERROR and WARN logs
filtered_df = df.filter(col("level").isin("ERROR", "WARN"))

# Count logs per module
logs_by_module = filtered_df.groupBy("module").agg(count("*").alias("log_count"))

# Count logs per day per level
logs_by_day_level = filtered_df.groupBy("date", "level").agg(count("*").alias("count"))

# Save output
logs_by_module.write.format("delta").mode("overwrite").save("dbfs:/mnt/data/logs_by_module")
logs_by_day_level.write.format("delta").mode("overwrite").save("dbfs:/mnt/data/logs_by_day_level")

